# Building a Simple Energy Model

The goal of this tutorial is to build a simple energy model using `MESSAGEix` with minimal features that can be expanded in future tutorials. We will build the model component by component, focusing on both the **how** (code implementation) and **why** (mathematical formulation).

# Follow Along at Home

The full model documentation is available online at [https://messageix.iiasa.ac.at](https://messageix.iiasa.ac.at)
    
<img src='assests/doc_page.png'>

And you can easily install `MESSAGEix` yourself and get all the tutorials:

```shell
$ conda install -c conda-forge message-ix
$ messageix-dl # install all tutorials to your current directory
```

TODO, brief primer on message as optimization minimum cost, parameters, constraints, etc.
vars are CAPS, pars are lower_case

# A Brave New World: Westeros with Electricity!

<table><tr><td><img src='assests/westeros.jpg' width='150'></td><td><img src='assests/base_res.png'></td></tr></table>

## Setup

First, we import all the packages we need.

In [14]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

The `Platform` is your connection to a database

In [2]:
mp = ix.Platform(dbtype='HSQLDB')

INFO:root:launching ixmp.Platform with local HSQLDB database at '/home/gidden/.local/ixmp/localdb/default'


Once connected, we make our `Scenario` which we use to build our model.

In [3]:
scenario = message_ix.Scenario(mp, model='Westeros Electrified', scen='baseline', version='new')

## Model Structure

We start by defining basic characteristics of the model, including time, space, and the energy system structure.

The model horizon will span 3 decades. Let's assume that we're far in the future after the events of A Song of Ice and Fire (~300 years after Aegon the conqueror).

| Math Notation | Model Meaning|
|---------------|--------------|
| $y \in Y$     | year         |

In [7]:
horizon = [700, 710, 720]
scenario.add_horizon({'year': horizon})

Our model will have a single `node`, i.e., its spatial dimension.


| Math Notation | Model Meaning|
|---------------|--------------|
| $n \in N$     | node         |

In [5]:
country = 'Westeros'
scenario.add_spatial_sets({'country': country})

And we fill in the energy system's `commodities`, `levels`, `technologies`, and `mode` (defining how certain technologies operate). 


| Math Notation | Model Meaning|
|---------------|--------------|
| $c \in C$     | commodity    |
| $l \in L$     | level        |
| $t \in T$     | technology   |
| $m \in M$     | mode         |

In [11]:
scenario.add_set("commodity", ["electricity", "light"])

scenario.add_set("level", ["secondary", "final", "useful"])

scenario.add_set("technology", ['coal_ppl', 'wind_ppl', 'grid', 'bulb'])

scenario.add_set("mode", "standard")

## Objective Function

The objective function seeks to minimize total discounted system cost over space and time. 



$\min_{n, y} \sum_{n,y \in Y^{M}} interestrate_{y} \cdot COST\_NODAL_{n,y}$


`COST_NODAL` is comprised of a variety of system costs defined later, but let's add the interest rate parameter.

In [10]:
rate = [0.05] * len(horizon)
unit = ['%'] * len(horizon)
scenario.add_par("interestrate", key=horizon, val=rate, unit=unit)

## Supply and Demand (or Balancing Commodities)

The fundamental premise of the model is to satisfy demand as cheaply as possible. We define a demand profile:

In [1]:
beta = 0.7 # an elasticity of demand
gdp = pd.Series([1., 1.21631, 1.4108], index=pd.Index(horizon, name='Time'))
demand_profile = gdp ** beta
demand_profile.plot(title='Demand')

NameError: name 'pd' is not defined

The `COMMODITY_BALANCE` equation ensures that `input`, `output`, and `demand` are met at each `level` in the energy system

$\begin{split}\sum_{\substack{n^L,t,m \\ y^V \leq y}} output_{n^L,t,y^V,y,m,n,c,l,h}
     \cdot & ACT_{n^L,t,y^V,y,m} \\
- \sum_{\substack{n^L,t,m, \\ y^V \leq y}} input_{n^L,t,y^V,y,m,n,c,l,h}
     \cdot & ACT_{n^L,t,m,y} \\
- \ demand_{n,c,l,y,h}
& \geq 0 \quad \forall \ l \in L\end{split}$

## Technologies

In [24]:
demand_per_year = 55209. / 8760 # from IEA statistics
elec_demand = pd.DataFrame({
        'node': country,
        'commodity': 'other_electricity',
        'level': 'useful',
        'year': horizon,
        'time': 'year',
        'value': demand_per_year * demand,
        'unit': 'GWa',
    })
scenario.add_par("demand", elec_demand)

demand_per_year = 6134. / 8760 # from IEA statistics
light_demand = pd.DataFrame({
        'node': country,
        'commodity': 'light',
        'level': 'useful',
        'year': horizon,
        'time': 'year',
        'value': demand_per_year * demand,
        'unit': 'GWa',
    })
scenario.add_par("demand", light_demand)

at.ac.iiasa.ixmp.exceptions.IxExceptionPyRaisable: at.ac.iiasa.ixmp.exceptions.IxException: The index set 'commodity' does not have an element 'other_electricity'!

### Engineering Parameters

In [ ]:
vintage_years, act_years = scenario.vintage_and_active_years()

In [ ]:
base_input = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

grid = pd.DataFrame(dict(
        technology = 'electricity_grid',
        level = 'secondary',
        value = 1.0,
        unit = '%',
        **base_input
        ))
scenario.add_par("input", grid)


bulb = pd.DataFrame(dict(
        technology = 'bulb',
        level = 'final',
        value = 1.0,
        unit = '%',
        **base_input
        ))
scenario.add_par("input", bulb)

cfl = pd.DataFrame(dict(
        technology = 'cfl',
        level = 'final',
        value = 0.3, 
        unit = '%',
        **base_input
        ))
scenario.add_par("input", cfl)

app = pd.DataFrame(dict(
        technology = 'appliances',
        level = 'final',
        value = 1.0,
        unit = '%',
        **base_input
        ))
scenario.add_par("input", app)

In [ ]:
make_df?

In [ ]:
base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}

imports = make_df(base_output, technology='import', commodity='electricity', 
                  level='secondary', value=1.)
scenario.add_par('output', imports)

grid = make_df(base_output, technology='electricity_grid', commodity='electricity', 
               level='final', value=0.873)
scenario.add_par('output', grid)

bulb = make_df(base_output, technology='bulb', commodity='light', 
               level='useful', value=1.)
scenario.add_par('output', bulb)

cfl = make_df(base_output, technology='cfl', commodity='light', 
              level='useful', value=1.)
scenario.add_par('output', cfl)

app = make_df(base_output, technology='appliances', commodity='other_electricity', 
              level='useful', value=1.)
scenario.add_par('output', app)

coal = make_df(base_output, technology='coal_ppl', commodity='electricity', 
               level='secondary', value=1.)
scenario.add_par('output', coal)

gas = make_df(base_output, technology='gas_ppl', commodity='electricity', 
              level='secondary', value=1.)
scenario.add_par('output', gas)

oil = make_df(base_output, technology='oil_ppl', commodity='electricity', 
              level='secondary', value=1.)
scenario.add_par('output', oil)

bio = make_df(base_output, technology='bio_ppl', commodity='electricity', 
              level='secondary', value=1.)
scenario.add_par('output', bio)

hydro = make_df(base_output, technology='hydro_ppl', commodity='electricity', 
                level='secondary', value=1.)
scenario.add_par('output', hydro)

wind = make_df(base_output, technology='wind_ppl', commodity='electricity', 
               level='secondary', value=1.)
scenario.add_par('output', wind)

solar_pv = make_df(base_output, technology='solar_pv_ppl', commodity='electricity', 
                   level='final', value=1.)
scenario.add_par('output', solar_pv)

In [ ]:
base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': horizon,
    'unit': 'y',
}

lifetimes = {
    'coal_ppl': 40,
    'gas_ppl': 30,
    'oil_ppl': 30,
    'bio_ppl': 30,
    'hydro_ppl': 60,
    'wind_ppl': 20,
    'solar_pv_ppl': 20,
    'bulb': 1,
    'cfl': 10,
}

for tec, val in lifetimes.items():
    df = make_df(base_technical_lifetime, technology=tec, value=val)
    scenario.add_par('technical_lifetime', df)

In [ ]:
base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}

capacity_factor = {
    'coal_ppl': 0.85,
    'gas_ppl': 0.75,
    'oil_ppl': 0.75,
    'bio_ppl': 0.75,
    'hydro_ppl': 0.5,
    'wind_ppl': 0.2,
    'solar_pv_ppl': 0.15,
    'bulb': 0.1, 
    'cfl':  0.1, 
}

for tec, val in capacity_factor.items():
    df = make_df(base_capacity_factor, technology=tec, value=val)
    scenario.add_par('capacity_factor', df)

### Technoeconomic Parameters

In [ ]:
base_inv_cost = {
    'node_loc': country,
    'year_vtg': horizon,
    'unit': 'USD/GWa',
}

# in $ / kW
costs = {
    'coal_ppl': 1500,
    'gas_ppl':  870,
    'oil_ppl':  950,
    'hydro_ppl': 3000,
    'bio_ppl':  1600,
    'wind_ppl': 1100,
    'solar_pv_ppl': 4000,
    'bulb': 5,
    'cfl':  900, 
}

for tec, val in costs.items():
    df = make_df(base_inv_cost, technology=tec, value=val * 1e6)
    scenario.add_par('inv_cost', df)

In [ ]:
base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}

# in $ / kW
costs = {
    'coal_ppl': 40,
    'gas_ppl':  25,
    'oil_ppl':  25,
    'hydro_ppl': 60,
    'bio_ppl':  30,
    'wind_ppl': 40,
    'solar_pv_ppl': 25,
}

for tec, val in costs.items():
    df = make_df(base_fix_cost, technology=tec, value=val * 1e6)
    scenario.add_par('fix_cost', df)

In [ ]:
base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

# in $ / MWh
costs = {
    'coal_ppl': 24.4,
    'gas_ppl':  42.4,
    'oil_ppl':  77.8,
    'bio_ppl':  48.2,
    'electricity_grid': 47.8,
}

for tec, val in costs.items():
    df = make_df(base_var_cost, technology=tec, value=val * 8760. * 1e3)
    scenario.add_par('var_cost', df)

## Dynamic Behavior Parameters

In [ ]:
base_growth = {
    'node_loc': country,
    'year_act': horizon[1:],
    'value': 0.05,
    'time': 'year',
    'unit': '%',
}

growth_technologies = [
    "coal_ppl", 
    "gas_ppl", 
    "oil_ppl", 
    "bio_ppl", 
    "hydro_ppl",
    "wind_ppl", 
    "solar_pv_ppl", 
    "cfl",
    "bulb",
]

for tec in growth_technologies:
    df = make_df(base_growth, technology=tec)
    scenario.add_par('growth_activity_up', df)

In [ ]:
base_initial = {
    'node_loc': country,
    'year_act': horizon[1:],
    'time': 'year',
    'unit': '%',
}

for tec in lights:
    df = make_df(base_initial, technology=tec, value=0.01 * light_demand['value'].loc[horizon[1:]])
    scenario.add_par('initial_activity_up', df)

In [ ]:
base_activity = {
    'node_loc': country,
    'year_act': [2010],
    'mode': 'standard',
    'time': 'year',
    'unit': 'GWa',
}

# in GWh - from IEA Electricity Output
activity = {
    'coal_ppl': 7184,
    'gas_ppl':  14346,
    'oil_ppl':  1275,
    'hydro_ppl': 38406,
    'bio_ppl':  4554,
    'wind_ppl': 2064,
    'solar_pv_ppl': 89,
    'import': 2340,
    'cfl': 0,
}

for tec, val in activity.items():
    df = make_df(base_activity, technology=tec, value=val / 8760.)
    scenario.add_par('bound_activity_up', df)
    scenario.add_par('bound_activity_lo', df)

In [ ]:
base_capacity = {
    'node_loc': country,
    'year_vtg': [2010],
    'unit': 'GWa',
}

cf = pd.Series(capacity_factor)
act = pd.Series(activity)
capacity = (act / 8760 / cf).dropna().to_dict()

for tec, val in capacity.items():
    df = make_df(base_capacity, technology=tec, value=val)
    scenario.add_par('bound_new_capacity_up', df)

In [ ]:
base_activity = {
    'node_loc': country,
    'year_act': horizon[1:],
    'mode': 'standard',
    'time': 'year',
    'unit': 'GWa',
}

# in GWh - base value from IEA Electricity Output
keep_activity = {
    'hydro_ppl': 38406,
    'bio_ppl':  4554,
    'import': 2340,
}

for tec, val in keep_activity.items():
    df = make_df(base_activity, technology=tec, value=val / 8760.)
    scenario.add_par('bound_activity_up', df)

## Emissions

In [ ]:
scenario.add_set("emission", "CO2")
scenario.add_cat('emission', 'GHGs', 'CO2')

In [ ]:
base_emissions = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'kg/kWa', # actually is tCO2/GWa
}

# units: tCO2/MWh
emissions = {
    'coal_ppl': ('CO2', 0.854),
    'gas_ppl':  ('CO2', 0.339),
    'oil_ppl':  ('CO2', 0.57),
}

for tec, (species, val) in emissions.items():
    df = make_df(base_emissions, technology=tec, emission=species, value=val * 8760. * 1000)
    scenario.add_par('emission_factor', df)

## Commit the datastructure and solve the model

In [ ]:
comment = 'initial commit for Austria model'
scenario.commit(comment)
scenario.set_as_default()

In [ ]:
scenario.solve('MESSAGE')

In [ ]:
scenario.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scenario, country)

In [ ]:
p.plot_new_capacity(baseyear=True, subset=plants)

In [ ]:
p.plot_new_capacity(baseyear=True, subset=lights)

In [ ]:
p.plot_capacity(baseyear=True, subset=plants)

In [ ]:
p.plot_capacity(baseyear=True, subset=lights)

In [ ]:
p.plot_demand(light_demand, elec_demand)

In [ ]:
p.plot_activity(baseyear=True, subset=plants)

In [ ]:
p.plot_activity(baseyear=True, subset=lights)

In [ ]:
p.plot_prices(baseyear=False, subset=['light', 'other_electricity'])

In [ ]:
mp.close_db()